<a href="https://colab.research.google.com/github/skimaza/assist/blob/main/cnn_application_deepface_assist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI 전략경영MBA 경영자를 위한 딥러닝 원리의 이해
# CNN 응용 - DeepFace 얼굴 인식 라이브러리 활용
## DeepFace 소스: https://github.com/serengil/deepface 

In [ ]:
from matplotlib import pyplot as plt

## google-api 오류때문에 http library downgrade 필요  
https://github.com/googleapis/google-api-python-client/issues/803  
비디오 인식 단계에서 필요

In [ ]:
!pip install httplib2==0.15.0

## DeepFace 설치
Colab에 기본으로 포함되지 않은 패키지는 shell command로 설치해야 함

In [ ]:
!pip install deepface

In [ ]:
from deepface import DeepFace

## deepface 소스코드를 가상머신에 복사
예제에 사용하는 이미지를 다운로드하기 위해 deepface 소스를 복사  
  
github의 코드를 복사하는 명령은 git clone

In [ ]:
!git clone https://github.com/serengil/deepface.git

## 예제에서 사용할 이미지가 다운로드되었는지 확인

In [ ]:
!ls

In [ ]:
!ls deepface/tests/dataset

# 데이터 이미지 확인

In [ ]:
import cv2

### img1.jpg 이미지 보기
deepface/tests/dataset 폴더 아래 img1.jpg 이미지 보기

In [ ]:
img1 = cv2.imread('deepface/tests/dataset/img1.jpg', cv2.IMREAD_COLOR) # 이미지 읽기

In [ ]:
img1.shape

In [ ]:
type(img1)

* 이미지는 3차원 numpy 배열로 표현된다.
* 3개의 차원은 각각 높이, 너비, 색상을 나타낸다.
* 색상 차원의 순서는 라이브러리마다 차이가 있을 수 있으므로 주의가 필요
    * matplotlib에서는 RGB, OpenCV (cv2)에서는 BGR이 디폴트임
    * 따라서 OpenCV로 읽은 데이터를 matplotlib에서 디스플레이할 때는 색상 차원의 순서를 변환해 줘야 한다 (아래 예제 참고)

### OpenCV로 읽은 것을 plt.imshow()로 display
이번 예제에서는 plt.imshow()로 이미지 디스플레이  
그대로 display하면 아래와 같이 보임

In [ ]:
plt.imshow(img1)

### BGR을 RGB로 변환하여 디스플레이

In [ ]:
img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
plt.imshow(img1_rgb)

In [ ]:
print(img1[2, 2, :])
print(img1_rgb[2,2,:]) # 첫번쨰와 세번째 값이 switch 되었다

이미지 크기가 바뀐게 아니라 plt.imshow에서 작게 디스플레이한 것임

In [ ]:
img1_rgb.shape

이제 img1은 필요없으므로 img1에 img1_rgb를 assign

In [ ]:
img1 = img1_rgb

### plt.imshow()를 써서 이미지를 디스플레이할 때 그림 크기를 키우기 위해서는 아래와 같은 방법을 쓴다.  
(주의) 마지막에 fig 변수를 close하지 않으면 garbage로 쌓이므로 주의.  
위의 예와 같이 plt.imshow만 사용하는 경우에는 Colab 셀의 마지막에 plt.show()를 생략해도 Colab에서 자동으로 호출해준다

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(img1_rgb)
plt.show()
plt.close(fig)

### 또는 아래와 같이 글로벌 default 설정을 바꿀 수도 있다

In [ ]:
plt.rcParams["figure.figsize"]

In [ ]:
plt.rcParams['figure.dpi'] # dot per inch

In [ ]:
plt.rcParams["figure.figsize"] = [12.0, 8.0]

In [ ]:
plt.imshow(img1)

# img2도 읽어보자

In [ ]:
img2 = cv2.imread('deepface/tests/dataset/img2.jpg', cv2.IMREAD_COLOR)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img2)

## 이미지를 읽어서 색상변환하는 함수를 정의

In [ ]:
def read_image(file, show=True):
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if show:
        plt.imshow(img)
        plt.show() # don't forget plt.show()
    return img

In [ ]:
img1 = read_image('deepface/tests/dataset/img1.jpg')

# 기본 라이브러리 import

In [ ]:
import os                # 운영체제 관련 기능을 제공하는 패키지
import glob              # 여러 파일이름을 읽어들이는 패키지
import numpy as np
from pathlib import Path # 파일 경로 처리를 도와주는 패키지
from natsort import natsorted # 파일이름을 일상에서 쓰는 순서로 정렬

## glob을 이용하여 dataset 폴더의 모든 파일리스트를 읽음

In [ ]:
glob.glob("deepface/tests/dataset/*.jpg")

파일의 순서가 예측할 수 없게 정렬되어 있음. (운영체제에 따라 달라짐)  
모든 파일리스트를 읽은 후에 natsorted를 이용하여 파일이름 순서로 정렬  
natsorted는 img2 보다 img11이 뒤에 온다는 것도 고려해서 정렬하는 natural order sort 함수

In [ ]:
imagefiles = natsorted(glob.glob("deepface/tests/dataset/*.jpg"))
print(imagefiles)

모든 이미지를 읽어들임  
all_titles에는 경로를 제외하고 파일이름만 저장

In [ ]:
all_images = []
all_titles = []
for file in imagefiles:
    all_titles.append(file.split('/')[-1])
    all_images.append(read_image(str(file), show=False))

In [ ]:
len(all_images)

In [ ]:
all_images[0].shape

In [ ]:
all_titles[:10]

여러 이미지를 그리드 형식으로 디스플레이하는 함수

In [ ]:
def show_img(img, ax=None, title=None):
    """Shows a single image."""
    if ax is None:
        ax = plt.gca()
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    if title:
        ax.set_title(title)

def show_img_grid(imgs, titles, size=3, clear_last=True):
    """Shows a grid of images."""
    n_images = len(imgs)
    n_grid = int(np.ceil(n_images**.5)) # grid size containing all images. ceiling sqrt(n_images) and convert to integer
    total_grids = n_grid * n_grid       # 이 함수는 행과 열의 길이가 같은 그리드를 구성하여 디스플레이
    _, axs = plt.subplots(n_grid, n_grid, squeeze=False, figsize=(size * n_grid, size * n_grid)) # squeeze=False makes axs always 2D grid
    for i, (img, title) in enumerate(zip(imgs, titles)):
        show_img(img, axs[i // n_grid][i % n_grid], title)
    if clear_last:
        if total_grids > n_images: # there're empty grid in the last row
            for i in range(n_images, total_grids):
                column = i - n_grid*(n_grid-1)
                axs[n_grid-1][column].axis('off')
    

In [ ]:
show_img_grid(all_images, all_titles)

# DeepFace 기능 확인

## img1과 img2가 동일인인지 확인하는 함수 verify

img1.jpg와 img2.jpg가 동일인인지 검증  
처음 실행하면 이미 학습된 결과를 다운로드한 후 실행

In [ ]:
result = DeepFace.verify("deepface/tests/dataset/img1.jpg", "deepface/tests/dataset/img2.jpg")

In [ ]:
result

두 이미지의 거리는 0.255, threshold를 0.4로 설정했으므로 두 이미지는 유사하다고 판단할 수 있다. 따라서 verified가 True

위 결과는 디폴트 모델인 'VGG-Face'를 사용한 것  
다른 모델도 적용할 수 있음  
model_name
        'VGG-Face', 
        'OpenFace', 
        'Facenet', 
        'Facenet512',
        'DeepFace',
        'DeepID',
        'Dlib',
        'ArcFace'

In [ ]:
result = DeepFace.verify("deepface/tests/dataset/img1.jpg", "deepface/tests/dataset/img2.jpg", model_name='Facenet')

In [ ]:
result

## 여러 이미지 중에 유사한 이미지 찾기

img1과 유사한 이미지

In [ ]:
df = DeepFace.find(img_path = "deepface/tests/dataset/img1.jpg", db_path = "deepface/tests/dataset")

In [ ]:
df

In [ ]:
jolie_files = df['identity'].to_list()
print(jolie_files)

In [ ]:
jolie_images = []
jolie_titles = []
for file in jolie_files:
    jolie_titles.append(file.split('/')[-1])
    jolie_images.append(read_image(str(file), show=False))

In [ ]:
show_img_grid(jolie_images, jolie_titles)

In [ ]:
df_aniston = DeepFace.find(img_path = "deepface/tests/dataset/img3.jpg", db_path = "deepface/tests/dataset")

In [ ]:
df_aniston

In [ ]:
aniston_files = df_aniston['identity'].to_list()
aniston_images = []
aniston_titles = []
for file in aniston_files:
    aniston_titles.append(file.split('/')[-1])
    aniston_images.append(read_image(str(file), show=False))

In [ ]:
show_img_grid(aniston_images, aniston_titles)

# Facial Emotion Recognition

### analyze 함수 사용
analyze 함수는 얼굴 이미지에서 얼굴 부분을 찾고 감정/나이/성별/인종 분류를 지원

In [ ]:
img8 = read_image("deepface/tests/dataset/img8.jpg")

In [ ]:
obj = DeepFace.analyze(img_path = "deepface/tests/dataset/img8.jpg", actions = ['age', 'gender', 'race', 'emotion'])

In [ ]:
obj

In [ ]:
obj.keys()

In [ ]:
r = obj['region']
h, w, x, y = r['h'], r['w'], r['x'], r['y']

In [ ]:
r

In [ ]:
import matplotlib.patches as patches

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img8)
rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()
plt.close(fig)

# 자신의 이미지로 테스트

# image capture from webcam

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

디폴트 face detector (opencv)는 얼굴을 놓치는 경우가 많다.  
detector는 mtcnn을 적용

In [ ]:
myobj = DeepFace.analyze(img_path = "photo.jpg", actions = ['age', 'gender', 'race', 'emotion'], detector_backend='mtcnn')

In [ ]:
myobj

In [ ]:
myimg = cv2.imread('photo.jpg')
myimg = cv2.cvtColor(myimg, cv2.COLOR_BGR2RGB)
plt.imshow(myimg)

In [ ]:
r = myobj['region']
h, w, x, y = r['h'], r['w'], r['x'], r['y']

In [ ]:
fig, ax = plt.subplots()
ax.imshow(myimg)
rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()
plt.close(fig)

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

myobj = DeepFace.analyze(img_path = "photo.jpg", actions = ['age', 'gender', 'race', 'emotion'], detector_backend='mtcnn')

myimg = cv2.imread('photo.jpg')
myimg = cv2.cvtColor(myimg, cv2.COLOR_BGR2RGB)

r = myobj['region']
h, w, x, y = r['h'], r['w'], r['x'], r['y']

fig, ax = plt.subplots()
ax.imshow(myimg)
rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()
plt.close(fig)

myobj

# 모델 살펴보기

In [ ]:
vgg = DeepFace.build_model('VGG-Face')

In [ ]:
vgg

## 모델 디스플레이를 위해 tensorflow import

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.utils.plot_model(vgg)

# Video recording from webcam

In [ ]:
from IPython.display import display, Javascript,HTML
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
def record_video(filename):
  js=Javascript("""
    async function recordVideo() {
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
       
      capture.textContent = "Start Recording";
      capture.style.background = "orange";
      capture.style.color = "white";
 
      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);
 
      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';
 
      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
     
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
 
      video.srcObject = stream;
      video.muted = true;
 
      await video.play();
 
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
 
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
 
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
       
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();
 
      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  try:
    display(js)
    data=eval_js('recordVideo({})')
    binary=b64decode(data)
    with open(filename,"wb") as video_file:
      video_file.write(binary)
    print(f"Finished recording video at:{filename}")
  except Exception as err:
    print(str(err))  

In [ ]:
video_path = "webcam1.mp4"
record_video(video_path)

In [ ]:
!ls

## 다음 함수는 짧은 비디오에서만 동작
길어지면 Colab에서 오류 발생  
예제에서는 1분 이내의 비디오이므로 문제없음

In [ ]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}" type="video/mp4"></video>""")
 

In [ ]:
show_video("webcam1.mp4")

In [ ]:
!ls -l

## 10 프레임 간격으로 이미지 추출

In [ ]:
images = []
cap = cv2.VideoCapture('webcam1.mp4')
i = 0
while True:
    ret, image = cap.read()
    if ret:
        if i % 10 == 0:
            img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(img_rgb)
        i += 1
    else:
        break

In [ ]:
len(images)

In [ ]:
plt.imshow(images[3])

### enforce_detection=False
얼굴 탐지에 실패해도 오류를 내지 않고 계속 진행  
전체 이미지를 입력으로 분석 수행

In [ ]:
results = DeepFace.analyze(img_path = images, actions = ['age', 'gender', 'race', 'emotion'], detector_backend='mtcnn', enforce_detection=False)

In [ ]:
results

In [ ]:
face_locations = []
emotions = []
for i, (inst, res) in enumerate(results.items()):
    print(i, res['age'], res['dominant_race'], res['gender'], res['dominant_emotion'])
    print(res['region'])
    face_locations.append([res['region']['x'], res['region']['y'], res['region']['w'], res['region']['h']])
    emotions.append(res['dominant_emotion'])

In [ ]:
face_locations

In [ ]:

def show_img2(img, rect, ax=None, title=None):
    """Shows a single image."""
    if ax is None:
        ax = plt.gca()
    #ax.imshow(img[...])
    ax.imshow(img)
    patch = patches.Rectangle(rect[:2], rect[2], rect[3], linewidth=3, edgecolor='g', facecolor="none")
    ax.add_patch(patch)
    ax.set_xticks([])
    ax.set_yticks([])
    if title:
        ax.set_title(title)

def show_img_grid2(imgs, rects, titles, size=3, clear_last=True):
    """Shows a grid of images."""
    n_images = len(imgs)
    n_grid = int(np.ceil(n_images**.5)) # grid size containing all images. ceiling sqrt(n_images) and convert to integer
    total_grids = n_grid * n_grid
    fig, axs = plt.subplots(n_grid, n_grid, squeeze=False, figsize=(size * n_grid, size * n_grid)) # squeeze=False makes axs always 2D grid
    for i, (img, rect, title) in enumerate(zip(imgs, rects, titles)):
        show_img2(img, rect, axs[i // n_grid][i % n_grid], title)
    if clear_last:
        if total_grids > n_images: # there're empty grid in the last row
            for i in range(n_images, total_grids):
                column = i - n_grid*(n_grid-1)
                axs[n_grid-1][column].axis('off')
    plt.show()
    plt.close(fig)

In [ ]:
def show_image_table2(imgs, boxes, titles=None, rows=3, cols=4,  figsize=(15,15)):
    total_pictures = rows * cols
    #boxes = [[b['x'], b['y'], b[]]]
    
    fig, axes = plt.subplots(rows, cols, squeeze=None, figsize=figsize)
    for i in range(rows):
        for j in range(cols):
            ind = i*cols + j
            #img = plt.imread(str(files[ind]))
            img = imgs[ind]
            axes[i,j].imshow(img)
            if boxes is not None:
                box = boxes[ind]
                rect = patches.Rectangle(box[:2], box[2], box[3], linewidth=3, edgecolor='g', facecolor='none')
                axes[i,j].add_patch(rect)
            axes[i,j].axis('off')
            if titles:
                axes[i,j].set_title(titles[ind])
            else:
                axes[i,j].set_title(str(ind))
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    plt.close(fig)

In [ ]:
show_image_table2(images, face_locations, titles=emotions) #, size=6)

## Face detector를 Retina face로 변경

In [ ]:
results = DeepFace.analyze(img_path = images, actions = ['age', 'gender', 'race', 'emotion'], detector_backend='retinaface', enforce_detection=False)

In [ ]:
face_locations = []
emotions = []
for i, (inst, res) in enumerate(results.items()):
    #print(i, res['age'], res['dominant_race'], res['gender'], res['dominant_emotion'])
    #print(res['region'])
    face_locations.append([res['region']['x'], res['region']['y'], res['region']['w'], res['region']['h']])
    emotions.append(res['dominant_emotion'])

show_image_table2(images, face_locations, titles=emotions) #, size=6)